In [1]:
from keras import optimizers
from keras import losses
from keras import metrics
from keras import regularizers
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from keras.preprocessing import sequence

import numpy as np
from numpy import argmax

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [121]:
# loading the data
import pandas as pd
data = pd.read_csv('data/SpeedDatingData.csv', engine='python')

In [122]:
# remove features that will not impact the match
data = data.drop(columns=['iid', 'id', 'gender', 'idg', 'condtn', 'wave', 'round', 'position', 'positin1'])
# remove features with not enough data
data = data.drop(columns=[i for i in data.columns if (data[i].count() < 6000)])

In [123]:
# drop rows with no data in 'match' column
data['match'] = pd.to_numeric(data['match'], errors='coerce')
data = data.dropna(subset=['match'])

# set aside labels as target vector
labels = pd.DataFrame([data['match']])
data = data.drop(columns=['match'])

# remove categorical features
cat_cols = [i for i in data.columns if data[i].dtype != 'float']
data = data.drop(columns=[i for i in cat_cols])

labels[0].apply(lambda x: int(x))
labels = labels.T

# oversampling data to improve class balance
from imblearn.over_sampling import SMOTE
data_resampled, labels_resampled = SMOTE().fit_resample(data, labels)

ModuleNotFoundError: No module named 'imblearn'

In [120]:
(labels == 1).sum()

match    1380
dtype: int64

In [105]:
# split the data into train and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.30)

In [106]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [107]:
print(y_train.shape)

(5864, 2)


In [108]:
val_size = int(np.round(len(x_train)*.1))
x_val = x_train[:val_size]
x_train_minus_val = x_train[val_size:]

y_val = y_train[:val_size]
y_train_minus_val = y_train[val_size:]

In [112]:
x_train_minus_val.shape

(5278, 93)

In [117]:
# train basic RNN
model = Sequential()
model.add(Dense(16, activation='relu', input_shape=(93,)))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax')) 

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train_minus_val,
                    y_train_minus_val,
                    epochs=10,
                    batch_size=216,
                    validation_data=(x_val, y_val))

Train on 5278 samples, validate on 586 samples
Epoch 1/10
5278/5278 [==============================] - 0s 88us/step - loss: 0.7040 - acc: 0.8350 - val_loss: 0.6472 - val_acc: 0.8549
Epoch 2/10
5278/5278 [==============================] - 0s 25us/step - loss: 0.6345 - acc: 0.8350 - val_loss: 0.6123 - val_acc: 0.8549
Epoch 3/10
5278/5278 [==============================] - 0s 26us/step - loss: 0.6027 - acc: 0.8350 - val_loss: 0.5791 - val_acc: 0.8549
Epoch 4/10
5278/5278 [==============================] - 0s 26us/step - loss: 0.5728 - acc: 0.8350 - val_loss: 0.5474 - val_acc: 0.8549
Epoch 5/10
5278/5278 [==============================] - 0s 25us/step - loss: 0.5444 - acc: 0.8350 - val_loss: 0.5176 - val_acc: 0.8549
Epoch 6/10
5278/5278 [==============================] - 0s 27us/step - loss: 0.5184 - acc: 0.8350 - val_loss: 0.4905 - val_acc: 0.8549
Epoch 7/10
5278/5278 [==============================] - 0s 25us/step - loss: 0.4963 - acc: 0.8350 - val_loss: 0.4679 - val_acc: 0.8549
Epoch 8/

In [91]:
cat_cols

['order',
 'partner',
 'match',
 'samerace',
 'dec_o',
 'field',
 'from',
 'zipcode',
 'career',
 'dec']